In [ ]:
!pip install -U adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.2 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html

In [ ]:
from transformers import AutoConfig, AutoModelWithHeads
import torch
from transformers import RobertaTokenizer
import transformers.adapters.composition as ac
import pandas as pd
from tqdm import tqdm
from scipy.stats import pearsonr
import random

In [ ]:
val_data = pd.read_csv('/content/drive/MyDrive/NLP_final_pro/data/messages_dev_features_ready_for_WS_2022.tsv', sep='\t')

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)

model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
""" load adapters """
empathy_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/EMP_emotion_stack"
distress_adapter_path = "/content/drive/MyDrive/NLP_final_pro/trained_adapters/DIS_emotion_stack"

empathy_adapter = model.load_adapter(empathy_adapter_path, load_as=empathy_adapter_path.split('/')[-1])
distress_adapter = model.load_adapter(distress_adapter_path, load_as=distress_adapter_path.split('/')[-1])

In [ ]:
model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']

In [ ]:
""" load pretrained adapter composition """

from transformers.adapters.composition import Fuse


fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-distress"

# load each individual adapter
dis_er_adapter = model.load_adapter(
                fusion_path + '/distress-emotional-reactions')
dis_ex_adapter = model.load_adapter(
                fusion_path + '/distress-explorations')
dis_ip_adapter = model.load_adapter(
                fusion_path + '/distress-interpretations')

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))

# # load head
# path, distress_adapter = model.load_head(fusion_path)

model.active_head = 'EpitomeFusion-distress'

In [ ]:
""" load pretrained adapter composition: empathy """

fusion_path = f"/content/drive/MyDrive/NLP_final_pro/trained_adapters/EpitomeFusion-empathy"

# load each individual adapter
emp_er_adapter = model.load_adapter(
                fusion_path + '/empathy-emotional-reactions'
            )
emp_ex_adapter = model.load_adapter(
                fusion_path + '/empathy-explorations'
            )
emp_ip_adapter = model.load_adapter(
                fusion_path + '/empathy-interpretations'
            )

# load adapter fusion
model.load_adapter_fusion(
    fusion_path
)

# set active adapters
model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))

# # load head
# path, empathy_adapter = model.load_head(fusion_path)

In [ ]:
model.active_head = 'EpitomeFusion-empathy'
model.active_head

In [ ]:
def predict_sen(sentence):
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokens)).to('cuda')

    # move the model to the GPU
    model.to('cuda')

    # set active adapters
    model.set_active_adapters(Fuse(dis_er_adapter, dis_ex_adapter, dis_ip_adapter))
    model.active_head = 'EpitomeFusion-distress'
    outputs = model(input_ids)
    dis_score_f = outputs.logits[0][0].tolist()

    # set active adapters
    model.set_active_adapters(Fuse(emp_er_adapter, emp_ex_adapter, emp_ip_adapter))
    model.active_head = 'EpitomeFusion-empathy'
    outputs = model(input_ids)
    emp_score_f = outputs.logits[0][0].tolist()

    # stack
    model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))
    model.active_head = ['EMP_emotion_stack', 'DIS_emotion_stack']
    outputs = model(input_ids)

    emp_score_s = outputs[0].logits[0][0].tolist()
    dis_score_s = outputs[1].logits[0][0].tolist()

    # move the model back to the CPU
    model.to('cpu')

    return emp_score_f, dis_score_f, emp_score_s, dis_score_s

In [ ]:
sen = val_data['essay'].tolist()
random_five = random.sample(sen, 5)

In [ ]:
for se in random_five:
  emp_score_f, dis_score_f, emp_score_s, dis_score_s = predict_sen(se)
  print(se)
  print("emp_score_f =",emp_score_f, "dis_score_f =",dis_score_f,"emp_score_s =", emp_score_s,"dis_score_s =" ,dis_score_s)

In [ ]:
re_sen = ["",
          "",
          "",
          "",
          ""]

In [ ]:
for re_sen in random_five:
  emp_score_f, dis_score_f, emp_score_s, dis_score_s = predict_sen(se)
  print(se)
  print("emp_score_f =",emp_score_f, "dis_score_f =",dis_score_f,"emp_score_s =", emp_score_s,"dis_score_s =" ,dis_score_s)